<a href="https://colab.research.google.com/github/Bread806/goldbach_backup_from_colab/blob/main/goldbach_1112_MLP_base2357_randomPrime_anchor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/myDrive")

Mounted at /content/myDrive


In [3]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LayerNormalization
from random import choice, sample


def is_prime(num):
    """檢查一個數字是否為質數"""
    if num <= 1:
        return False
    sqr_num = int(num ** 0.5)
    # for i in range(2, sqr_num+ 1):
    for i in range(2, sqr_num+ 1):
        if num % i == 0:
            return False
    return True


def prime_table(x):
    """建立小於x的質數表"""
    primes = [num for num in range(2, x) if is_prime(num)]
    return primes


def convert_base_into_list(number, base, width=10):
    result = []  # init list

    for i in range(width):
        result.append(number % base)
        number = number // base
    result = result[::-1]
    return result


def prime_to_index(primeSize, primes, number):
    for index in range(primeSize):
        if number == primes[index]:
            return index
    return -1

def add_prime(primeTable, limit, addSize=5):
    halfList = [num for num in primeTable if num <= int(limit//2)]
    if len(halfList) < 5:
      halfList += [0] * (5 - len(halfList))
    return sample(halfList, addSize)

def add_aim(index, staticFlagX):

  #feature = [index-flagN for flagN in staticFlagX] + []
  return [index-flagN for flagN in staticFlagX]


In [ ]:
# shuffle

print("---loading data---")
# load data
trainData = pd.read_csv('/content/myDrive/MyDrive/實驗/goldbach/csv/traindata_min_size_1000.csv')
trainDataNumbers = trainData['Number'].values
trainDataLabel = trainData['Factors'].values
print("---loading data done.---")

# shuffle training data
shuffleIndices = np.random.permutation(len(trainDataNumbers))
shuffledNumbers = trainDataNumbers[shuffleIndices]
shuffledLabel = trainDataLabel[shuffleIndices]

---loading data---
---loading data done.---


In [ ]:
print (shuffledNumbers[5])
print (shuffledLabel [5])

3030248
11418


In [ ]:
# loadding prime table
primeTable = pd.read_csv("/content/myDrive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


In [ ]:

def add_prime(primeTable, limit, addSize=5):

    halfList = [num for num in primeTable if num <= int(limit//2)]
    if len(halfList) < 5:
            halfList += [0] * (5 - len(halfList))
    return sample(halfList, addSize)

#limit = 8
print (add_prime(primeTableNumber, 22, addSize=5))

[2, 7, 11, 5, 3]


In [ ]:
"""
feature  base2357 + 隨機小於數字n的五個質數 + anchor（n與固定數字的差距量）
"""
# collect data
# prepare training set
mergedTrainNumber = []
mergedTrainLabel = []
staticFlagX = [316978, 2652438, 3897236, 3226506, 3571052, 3276150]
staticFlagY = [1577, 20090, 15400, 22108, 11879, 29621]

for num in shuffledNumbers:

    # base(2,3,5,7), aim, radom choice 5 in half of prime
    feature = []
    itself = [num]
    feature = convert_base_into_list(num, 2)+convert_base_into_list(num, 3)+convert_base_into_list(num, 5)+convert_base_into_list(num, 7) + add_prime(primeTableNumber, num, 5)+ add_aim(num, staticFlagX) + staticFlagX + staticFlagY
    #print (feature)
    mergedTrainNumber.append(feature)
    print(num, " covert to base 2 3 5 7 is done.")

for num in shuffledLabel:
    mergedTrainLabel.append(num)

    print("merge number ", num, " is done.")

#print (mergedTrainNumber)

In [ ]:
# # normalization
# mergedTrainNumber = tf.cast(mergedTrainNumber , tf.float32)
# mean = tf.reduce_mean(mergedTrainNumber)
# sttdev = tf.math.reduce_std(mergedTrainNumber)
# normalizedArray = (mergedTrainNumber - mean) / sttdev

mergedArray = np.array(mergedTrainNumber)
mergedLabel = np.array(mergedTrainLabel)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(mergedArray, mergedLabel, test_size=0.2, random_state=42)

In [ ]:
# start training
# 清空所有
keras.clear_session()

################################################################

# require gpu
device = "/device:GPU:0"
with tf.device(device):
    print(f"--- gpu {device} is loaded. ---")
    model = Sequential()

    model.add(Dense(units=63, input_shape=(63,), activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=1, activation='linear'))

    print(model.summary())

    custom_adam_optimizer = Adam(learning_rate=0.001)
    model.compile(loss='MSE', optimizer=custom_adam_optimizer,
                  metrics=['accuracy'])    # train version 2

    # train model
    print("---starting training---")
    trainHistory = model.fit(X_train, y_train, epochs=200,batch_size=2048, validation_data=(X_val, y_val))
    print("--------------------------------------------------------")

    # save model
    model.save('./csv/goldbach_model_1112_400w.h5')
print("model saved.")


## predict

In [1]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [4]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


In [5]:
## load data
#predict

from tensorflow.keras.models import load_model
import csv
import pandas as pd
import numpy as np

# 指定 HDF5 模型文件的路徑
modelPath = '/content/Drive/MyDrive/實驗/model_save/goldbach_model_1112_400w.h5'

# 使用 load_model
model = load_model(modelPath)

# load test_set and prime data
testDataPath = '/content/Drive/MyDrive/實驗/goldbach/csv/test_set.csv'
testData = pd.read_csv(testDataPath)
testDataNumbers = testData['Number'].values
testDataLabel = testData['Factors'].values
print("---loading test data done.---")

---loading test data done.---


In [6]:
# shuffle training data
shuffleIndices = np.random.permutation(len(testDataNumbers))
shuffledNumbers = testDataNumbers[shuffleIndices]
shuffledLabel = testDataLabel[shuffleIndices]

In [7]:
# collect data
# prepare training set
mergedTestNumber = []
mergedTestLabel = []
staticFlagX = [316978, 2652438, 3897236, 3226506, 3571052, 3276150]
staticFlagY = [1577, 20090, 15400, 22108, 11879, 29621]

for num in shuffledNumbers:

    # base(2,3,5,7), aim, radom choice 5 in half of prime
    feature = []
    itself = [num]
    feature = convert_base_into_list(num, 2)+convert_base_into_list(num, 3)+convert_base_into_list(num, 5)+convert_base_into_list(num, 7) + add_prime(primeTableNumber, num, 5)+ add_aim(num, staticFlagX) + staticFlagX + staticFlagY
    #print (feature)
    mergedTestNumber.append(feature)
    print(num, " covert to base 2 3 5 7 is done.")

for num in shuffledLabel:
    mergedTestLabel.append(num)

    print("merge number ", num, " is done.")

#print (mergedTrainNumber)

4550968  covert to base 2 3 5 7 is done.
4264218  covert to base 2 3 5 7 is done.
4170488  covert to base 2 3 5 7 is done.
4769998  covert to base 2 3 5 7 is done.
4628068  covert to base 2 3 5 7 is done.
4634032  covert to base 2 3 5 7 is done.
4215008  covert to base 2 3 5 7 is done.
4010462  covert to base 2 3 5 7 is done.
4817322  covert to base 2 3 5 7 is done.
4263072  covert to base 2 3 5 7 is done.
4874976  covert to base 2 3 5 7 is done.
4870030  covert to base 2 3 5 7 is done.
4158576  covert to base 2 3 5 7 is done.
4704764  covert to base 2 3 5 7 is done.
4411002  covert to base 2 3 5 7 is done.
4282102  covert to base 2 3 5 7 is done.
4477430  covert to base 2 3 5 7 is done.
4291942  covert to base 2 3 5 7 is done.
4264618  covert to base 2 3 5 7 is done.
4151108  covert to base 2 3 5 7 is done.
4657756  covert to base 2 3 5 7 is done.
4306378  covert to base 2 3 5 7 is done.
4023648  covert to base 2 3 5 7 is done.
4010118  covert to base 2 3 5 7 is done.
4604856  covert 

In [8]:
mergedArray = np.array(mergedTestNumber)
mergedLabel = np.array(mergedTestLabel)

In [9]:
evaluation = model.evaluate(mergedArray, mergedLabel)
XNewData = testDataNumbers, testDataLabel
print ("Loss:", evaluation[0])
print ("Accuracy:", evaluation[1])
print ("##########################")
predictions = model.predict(mergedArray)

2/2 [==============================] - 0s 11ms/step - loss: 54779140.0000 - accuracy: 0.0000e+00
Loss: 54779140.0
Accuracy: 0.0
##########################
2/2 [==============================] - 0s 6ms/step


In [10]:
print (f"測試五十筆資料")
roundPredictions = np.round(predictions)
for index, number in enumerate(testDataNumbers):
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案為:{testDataLabel[index]}\t模型預估為{predictions[index]}, 四捨五入到{roundPredictions[index]}")


測試五十筆資料
第0筆資料為數字:	4234636	|正確答案為:16566	模型預估為[22664.182], 四捨五入到[22664.]
第1筆資料為數字:	4010118	|正確答案為:31717	模型預估為[21441.32], 四捨五入到[21441.]
第2筆資料為數字:	4291942	|正確答案為:14594	模型預估為[21118.662], 四捨五入到[21119.]
第3筆資料為數字:	4151108	|正確答案為:14832	模型預估為[23718.107], 四捨五入到[23718.]
第4筆資料為數字:	4914148	|正確答案為:15673	模型預估為[23024.959], 四捨五入到[23025.]
第5筆資料為數字:	4634032	|正確答案為:16346	模型預估為[23052.361], 四捨五入到[23052.]
第6筆資料為數字:	4010462	|正確答案為:13108	模型預估為[21274.857], 四捨五入到[21275.]
第7筆資料為數字:	4665262	|正確答案為:18098	模型預估為[20329.607], 四捨五入到[20330.]
第8筆資料為數字:	4874538	|正確答案為:31284	模型預估為[23971.713], 四捨五入到[23972.]
第9筆資料為數字:	4867410	|正確答案為:42050	模型預估為[21465.465], 四捨五入到[21465.]
第10筆資料為數字:	4769998	|正確答案為:15623	模型預估為[24177.658], 四捨五入到[24178.]
第11筆資料為數字:	4550968	|正確答案為:15886	模型預估為[24126.963], 四捨五入到[24127.]
第12筆資料為數字:	4324986	|正確答案為:28364	模型預估為[20886.908], 四捨五入到[20887.]
第13筆資料為數字:	4306378	|正確答案為:14047	模型預估為[23443.396], 四捨五入到[23443.]
第14筆資料為數字:	4657756	|正確答案為:14997	模型預估為[22157.357], 四捨五入到[22157.]
第15筆資料為數字:	4948220	|正確答案為:22961	模型預估為[21532

In [11]:
print ("\t正確答案  "  , "\t/" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t/  \t{roundPredictions[index]} = {testDataLabel[index]/roundPredictions[index]} ")
print ("###########################################################################")

print ("\t正確答案  "  , "\t-" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t-  \t{roundPredictions[index]} = {testDataLabel[index]-roundPredictions[index]} ")

	正確答案   	/   	模型預測
0	16566  		/  	[22664.] = [0.7309389] 
1	31717  		/  	[21441.] = [1.4792687] 
2	14594  		/  	[21119.] = [0.6910365] 
3	14832  		/  	[23718.] = [0.62534785] 
4	15673  		/  	[23025.] = [0.6806949] 
5	16346  		/  	[23052.] = [0.7090925] 
6	13108  		/  	[21275.] = [0.6161222] 
7	18098  		/  	[20330.] = [0.8902115] 
8	31284  		/  	[23972.] = [1.3050225] 
9	42050  		/  	[21465.] = [1.959003] 
10	15623  		/  	[24178.] = [0.6461659] 
11	15886  		/  	[24127.] = [0.6584325] 
12	28364  		/  	[20887.] = [1.3579738] 
13	14047  		/  	[23443.] = [0.59919804] 
14	14997  		/  	[22157.] = [0.6768516] 
15	22961  		/  	[21532.] = [1.0663663] 
16	16070  		/  	[22463.] = [0.71539867] 
17	13302  		/  	[21605.] = [0.6156908] 
18	13996  		/  	[21457.] = [0.6522813] 
19	15871  		/  	[20974.] = [0.7566988] 
20	15845  		/  	[23193.] = [0.6831803] 
21	32717  		/  	[21697.] = [1.5079043] 
22	31204  		/  	[20448.] = [1.5260172] 
23	19410  		/  	[20373.] = [0.95273155] 
24	30918  		/  	[22844.] = [

In [13]:
def calculate_mse(predictions, targets):
    """
    計算均方誤差 (MSE)

    參數:
    predictions: 預測值的列表或陣列
    targets: 實際值的列表或陣列，與預測值對應

    返回值:
    mse: 均方誤差
    """
    # 確保預測值和實際值的長度相等
    if len(predictions) != len(targets):
        raise ValueError("預測值和實際值的長度不一致")

    # 計算平方誤差
    squared_errors = [(p - t) ** 2 for p, t in zip(predictions, targets)]

    # 計算均方誤差
    mse = sum(squared_errors) / len(predictions)

    return mse

# 使用 LN
mse_result = calculate_mse(roundPredictions, testDataLabel)

# 使用 MLP(old)
#mse_result = calculate_mse(predictions, testDataPartition)

print("均方誤差 (MSE):", mse_result)


均方誤差 (MSE): [62020568.]


In [15]:
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Number   50 non-null     int64
 1   Factors  50 non-null     int64
dtypes: int64(2)
memory usage: 928.0 bytes


In [17]:
import math
mean = testData['Factors'].mean()

#+N MSE
MSE = int(calculate_mse(roundPredictions, testDataLabel))

#MLP MSE
# MSE = int(calculate_mse(predictions, testDataPartition))

RMSE= math.sqrt(MSE)
errorRate = RMSE/mean

print (f"MSE : {MSE}\nRMSE : {RMSE}\nError Rate : {errorRate}  ->  {round(errorRate,7)*100}%")

MSE : 62020568
RMSE : 7875.3138350163545
Error Rate : 0.37434611280304686  ->  37.43461%


<ipython-input-17-b1f4c19fa9ab>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  MSE = int(calculate_mse(roundPredictions, testDataLabel))
